# Spark Optimization


1. `Application` level Optimization 
    - `code` optimization
    - use of `cache`
    - use of `reduceByKey` instead of `groupbykey`
 
2. `Cluster` level Optimization 
    - how spark execute our code
    - how resources are allocated across executor/container/JVM
    - how tasks are executed, etc. 

# Executor 

## At a high level:

Our intention is always to make sure that our `job` gets the right amount of `resources`
   - Its a container of resources (`CPU` and `Memory`)
   - 1 Node can hold more than 1 executor 
   - `Container`/`Executor`/`JVM` -> All are equivalent in Spark 

Lets say we have 'c' no. of cores and ''x' GB of meomory in each machine. Then,
   - we can have (c-1 cores and x-1 GB of memory) which we can use for executors 
   - 1 core and 1 GB memory is used for background process (deamon threads) 

## Two strategies we can use while creating the executors 

   - `Thin` **Executor** : 
        - Intention is to create more executors with each executor holding `min` possible resources
        - If a node is having 16 core CPU and 64GB memory, we can have max 16 executors with 4GB memory each 
        - We **can not have multi-threading** as each executor is having only 1 CPU core 
        - In case of `Broadcast/Shared Variables` we need to **copy the same data for multiple executors.** 
        - This type of executor is not that good 
        
        
   - `Fat` **Executor** :
        - Intention is to give `max` resources to each executor 
        - If a node is having 16 core CPU and 64GB memory, we can have 1 single executor with all its CPU and Memory
        - We can **have multithreading, but lot of multithreading is also not recommended**, we should have a balanced approch 
        - It is observed that, if the executor is holding more than 5 CPU Cores then HDFS throughput suffers 
        - If the executor holds very large amount of memory, then GC collection may take a lot of time. 

# Recommendation 
What would be the best way to operate, assume **each worker node have (16 Core CPU and 64 GB memory):**

#### What we want ?
   - We want multi-threading within a executor 
   - At the same time we want the HDFS throughput NOT to suffer 
   
#### How can we go from here ?
   - 5 is the right choice of no. of CPU cores within each executor 
   - So, if we have 16 Core CPU and 64 GB memory in a node, then **within each node**
       - we can have 3 executor
       - within this 21GB RAM some part of it will go to `Overhead (Off Heap Memory)`
           - `Overhead (Off Heap Memory)` : The memory which is not part of the JVM (its RAW memory)
           - How much is that ? Its `max(384MB, 7% of executor memory) = 1.5GB` (Overhead/Off Heap memory) 
           - This memory **wont be part of the executor/container**
           - So, **each executor would have effectively 21-1.5 ~ 19.5GB memory** 
       - So, ultimately **each executor would have 5 CPU cores and 19.5 GB memory**

![mem_arch](../img/mem_arch.png)

#### Now, lets assume we have a 10 Node cluster (Worker Nodes) 

  - Then we will have **30 Executors (10*3)**
  - Each executor would have **5 CPU cores and ~19.5GB memory**
  - Out of this 30 Executor, **1 executor would be taken by YARN Application**, so effectively we would have **29 executors**
  
#### Also, if we summarise, 
 - we know that the `number of tasks created` will be **equal to** the `number of partitions (of our data)`
 - but the `number of tasks executed in parallel` will **depend on** the `number of CPU cores available in each executor`. 

In [ ]:
10 Partition 
10 tasks ===> We have 1 Excutor with only 2 CPU (10 tasks in paraller (this will happen)) 

1, 2 ====> 3, 4 ====> 